In [68]:
import os
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

def load_data(file):
    return pd.read_csv(file)

path = "datas/"
general_data = load_data(path +"general_data.csv")
employee_survey_data = load_data(path +"employee_survey_data.csv")
manager_survey_data = load_data(path +"manager_survey_data.csv")
in_time = load_data(path +"in_time.csv")
out_time = load_data(path +"out_time.csv")
#general_data.info()
test= general_data.drop(columns=['Attrition','BusinessTravel'],axis=1).select_dtypes(include=["object"]).keys()
general_data
print(test)

Index(['Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus',
       'Over18'],
      dtype='object')


In [69]:
def to_datetime(strdate):
    if type(strdate) == str:
        return datetime.strptime(strdate, "%Y-%m-%d %H:%M:%S").timestamp()
    else:
        return 0

def get_working_time(in_time_data, out_time_data):
    out_time_df = (
        out_time_data
        .iloc[:,1:]
        .applymap(lambda x: to_datetime(x))
    )
    in_time_df = (
        in_time_data
        .iloc[:,1:]
        .applymap(lambda x: to_datetime(x))
    )
    hours_per_day = out_time_df - in_time_df
    in_time_data["MeanWorkingHours"] = hours_per_day.mean(axis = 1)/3600
    return in_time_data[["EmployeeID", "MeanWorkingHours"]]

In [70]:
class MergeDataFrame(BaseEstimator, TransformerMixin):
    def __init__(self, employee_survey_data, manager_survey_data, in_time, out_time):
        self.employee_survey_data = employee_survey_data
        self.manager_survey_data = manager_survey_data
        self.in_time= in_time
        self.out_time = out_time
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.join(self.employee_survey_data.set_index("EmployeeID"), on="EmployeeID")
        X = X.join(self.manager_survey_data.set_index("EmployeeID"), on="EmployeeID")
        X = pd.merge(X, get_working_time(in_time, out_time), on='EmployeeID', how='inner')

        return X

In [71]:
class DeleteUnEthicColumn(BaseEstimator, TransformerMixin):
    def __init__(self, array):
        self.array = array

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        for i in self.array:
            X.drop(i, axis=1, inplace=True)
        return X


In [72]:
class OrgDataFrame(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X["Attrition"] = X["Attrition"].replace(to_replace=['No', 'Yes'], value=[-1, 1])
        X["BusinessTravel"] = X["BusinessTravel"].replace(to_replace=['Non-Travel', 'Travel_Rarely', 'Travel_Frequently'], value=[1, 2, 3])
        return X


In [73]:
'''class OneHotDataFrame(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        one_hot_encoder = OneHotEncoder()
        object_attribs= X.select_dtypes(include=["object"]).keys()

        encoded_datas_array = one_hot_encoder.fit_transform(object_attribs).toarray()
        encoded_datas_labels = one_hot_encoder.categories_
        encoded_datas_labels = np.hstack([
        encoded_data = pd.DataFrame(encoded_datas_array, columns=encoded_datas_labels)
        #encoded_data["EmployeeID"] = datas["EmployeeID"]


        X = X.join(encoded_data)
        X.drop(['EducationField', 'Department', "JobRole"], axis=1, inplace=True, errors="ignore")'''




'class OneHotDataFrame(BaseEstimator, TransformerMixin):\n    def fit(self, X, y=None):\n        return self\n\n    def transform(self, X, y=None):\n        one_hot_encoder = OneHotEncoder()\n        object_attribs= X.select_dtypes(include=["object"]).keys()\n\n        encoded_datas_array = one_hot_encoder.fit_transform(object_attribs).toarray()\n        encoded_datas_labels = one_hot_encoder.categories_\n        encoded_datas_labels = np.hstack([\n        encoded_data = pd.DataFrame(encoded_datas_array, columns=encoded_datas_labels)\n        #encoded_data["EmployeeID"] = datas["EmployeeID"]\n\n\n        X = X.join(encoded_data)\n        X.drop([\'EducationField\', \'Department\', "JobRole"], axis=1, inplace=True, errors="ignore")'

In [74]:
class ScaleData(BaseEstimator, TransformerMixin):
    def __init__(self, array):
        self.array = array

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        std_scale = StandardScaler().fit()
        X_scaled = pd.DataFrame(std_scale.transform(X), columns=X.columns)
        return X_scaled

In [124]:
def build_pipeline(X):

#Pipeline one for preparation
    #Merge all files in one and change in and out datetime
    prepa_pipeline= Pipeline([
        ("merge", MergeDataFrame(load_data(path +"employee_survey_data.csv"),
                                 load_data(path +"manager_survey_data.csv"),
                                 load_data(path +"in_time.csv"),
                                 load_data(path +"out_time.csv"))),
        #delete useless columns and unEthic
        ("delete", DeleteUnEthicColumn([
                                        "Age",
                                        "Over18",
                                        "Gender",
                                        "MaritalStatus",
                                        "EmployeeCount",
                                        "StandardHours",
                                        "EmployeeID"
        
        ])),
        #encode Attrition and BusinessTravel columns
        ("label",OrgDataFrame()),
    ])
    #create a DataFrame with pipeline 
    data_pipeline = prepa_pipeline.fit_transform(X)
    ###print(data_pipeline)
    #create label for classification
    y = data_pipeline['Attrition']
    
    #list all columns who are object to onehot encode them
    object_attribs= data_pipeline.drop(columns=['Attrition','BusinessTravel'],axis=1).select_dtypes(include=["object"]).keys()
    ###print(type(object_attribs))
    ###num_search = data_pipeline.drop(columns=['Attrition'],axis=1).select_dtypes(include=[np.number])
    #list all columns who are number to scale them
    num_search = data_pipeline.select_dtypes(include=[np.number])
    num_attribs = list(num_search)
    #print(object_attribs)
    #print(num_attribs)

    #create onehot encode step pipeline   
    onehot_pipeline = Pipeline(steps=[
        ('onehot',OneHotEncoder())
    ])
    #create scale step pipeline
    scale_pipeline = Pipeline(steps=[
        ('std_scaler', StandardScaler()),
    ])
    #create column transformers 
    pipeline = ColumnTransformer(transformers=[
        ("numerisation",scale_pipeline,num_attribs),
        ("one", onehot_pipeline, object_attribs) 
        ],
        remainder='drop',
        n_jobs=-1)

    #create final pipeline
    full_pipeline = Pipeline(steps=[
        ("pipeline", pipeline),
    ])

    #print pipeline
    from sklearn import set_config
    set_config(display='diagram')
    display(full_pipeline)
    
    #execute pipeline
    data_fullpipeline = full_pipeline.fit_transform(data_pipeline)
    


    #get all columns name to recreate index on dataFrame
    cols_names=full_pipeline.named_steps["pipeline"].named_transformers_["one"].named_steps["onehot"].get_feature_names_out(object_attribs)
    all_cols=np.concatenate([num_attribs,cols_names])
    data_fullpipeline=pd.DataFrame(data_fullpipeline,columns=all_cols)
    
    #create unlabeled dataset
    X = data_fullpipeline.drop(columns=['Attrition'],axis=1)

    
    return data_fullpipeline, X, y

In [127]:
data, X, y = build_pipeline(general_data)

Pipeline(steps=[('pipeline',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('numerisation',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['Attrition',
                                                   'BusinessTravel',
                                                   'DistanceFromHome',
                                                   'Education', 'JobLevel',
                                                   'MonthlyIncome',
                                                   'NumCompaniesWorked',
                                                   'PercentSalaryHike',
                                                   'StockOptionLevel',
                                                   'TotalWorkingYears',
                                                   'TrainingTimesLastYear',
                                                   'YearsAtCompany',
                                                   'YearsSinceLastPromotion',
                                                   'YearsWithCurrManager',
                                                   'EnvironmentSatisfaction',
                                                   'JobSatisfaction',
                                                   'WorkLifeBalance',
                                                   'JobInvolvement',
                                                   'PerformanceRating',
                                                   'MeanWorkingHours']),
                                                 ('one',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  Index(['Department', 'EducationField', 'JobRole'], dtype='object'))]))])

In [128]:
data

,Attrition,BusinessTravel,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,...,EducationField_Technical Degree,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative
0,-0.438422,-0.162399,-0.393938,-0.891688,-0.961486,1.405136,-0.678311,-1.150554,-0.932014,-1.321101,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.280906,1.717339,0.099639,-1.868426,-0.961486,-0.491661,-1.078535,2.129306,0.241988,-0.678538,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.438422,1.717339,0.963398,1.061787,1.749610,2.725053,-0.678311,-0.057267,2.589994,-0.807051,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.438422,-2.042138,-0.887515,2.038524,0.845911,0.386301,0.122136,-1.150554,2.589994,0.221050,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.438422,-0.162399,0.099639,-1.868426,-0.961486,-0.884109,0.522360,-0.877232,1.415991,-0.293001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,-0.438422,-0.162399,-0.517332,1.061787,-0.961486,-0.100700,0.122136,0.489376,0.241988,-0.164488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4406,-0.438422,-0.162399,-0.887515,1.061787,-0.961486,-0.812504,-0.278088,-0.057267,-0.932014,-0.164488,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4407,-0.438422,-0.162399,1.950552,-0.891688,-0.057788,-0.595138,-1.078535,1.309341,-0.932014,-0.807051,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4408,-0.438422,-0.162399,1.086793,-0.891688,-0.961486,-0.872210,-1.078535,-0.330589,0.241988,-0.164488,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
X    

,BusinessTravel,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,...,EducationField_Technical Degree,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative
0,-0.162399,-0.393938,-0.891688,-0.961486,1.405136,-0.678311,-1.150554,-0.932014,-1.321101,2.483396,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.717339,0.099639,-1.868426,-0.961486,-0.491661,-1.078535,2.129306,0.241988,-0.678538,0.155707,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.717339,0.963398,1.061787,1.749610,2.725053,-0.678311,-0.057267,2.589994,-0.807051,-0.620189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-2.042138,-0.887515,2.038524,0.845911,0.386301,0.122136,-1.150554,2.589994,0.221050,1.707500,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.162399,0.099639,-1.868426,-0.961486,-0.884109,0.522360,-0.877232,1.415991,-0.293001,-0.620189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,-0.162399,-0.517332,1.061787,-0.961486,-0.100700,0.122136,0.489376,0.241988,-0.164488,1.707500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4406,-0.162399,-0.887515,1.061787,-0.961486,-0.812504,-0.278088,-0.057267,-0.932014,-0.164488,-0.620189,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4407,-0.162399,1.950552,-0.891688,-0.057788,-0.595138,-1.078535,1.309341,-0.932014,-0.807051,0.931603,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4408,-0.162399,1.086793,-0.891688,-0.961486,-0.872210,-1.078535,-0.330589,0.241988,-0.164488,-0.620189,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
y

0      -1
1       1
2      -1
3      -1
4      -1
       ..
4405   -1
4406   -1
4407   -1
4408   -1
4409   -1
Name: Attrition, Length: 4410, dtype: int64